<a href="https://colab.research.google.com/github/Develop-It-19/IMDB_Sentiment_Analysis_ULMFit_AWD_LSTM/blob/master/DL_2019_ULMFiT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies and Create DataBunch

In [0]:
#Training an AWD-LSTM Model using a Transfer Learning approach called ULMFiT (Universal Language Model Fine-tuning for Text Classification) with IMDb Sample Dataset for Sentiment Analysis.

In [0]:
from fastai.text import *

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [5]:
!ls {path}

texts.csv


In [6]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab = data_lm.train_ds.vocab, bs = 32)

In [8]:
data_lm.show_batch()

idx,text
0,"savage killer , melting every step of the way , he is xxunk by his friend , xxmaj dr. xxmaj xxunk xxmaj nelson ( xxmaj burr debenning ) . \n \n xxmaj this is often so xxunk funny - with enough absurd lines and situations to go around - that it 's hard for me to believe that the laughs are all xxunk . xxmaj it seems to me"
1,"xxmaj one becomes a character in a xxmaj king story they read , whereas xxmaj one is limited to petty xxunk of that same character when filmed . xxmaj for as long as xxmaj king writes , xxmaj hollywood will try shooting everything that comes out of his word xxunk , without any regard to whether or not they should . i do n't blame the filmmakers for trying ,"
2,"happen and i had to agree . xxmaj the xxmaj scene xxmaj that xxmaj explains xxmaj all was adequate and managed to explain all of the questions and mysterious dialogue bits throughout the movie but we were just checking them off a list . ( "" xxmaj oh , okay , that 's why xxmaj brad had that happen and xxmaj jonathan says this and ... "" ) \n \n"
3,"send this xxmaj uli xxmaj xxunk guy back to xxup movie xxup school . xxmaj who ever told him xxup he knew xxup how to make a movie ? xxmaj can just xxup anybody make movies these days ? xxmaj in the past , it always xxup required xxup talent before someone could make a movie . xxmaj after watching this lame xxup btk movie and the others he 's"
4,"training xxmaj day or xxmaj king xxmaj arthur , you will be impressed with the birth of this director in his second film xxmaj bait . xxmaj while xxmaj foxx gives a horrid , xxunk performance , there are certain scenes , which define xxmaj fuqua and demonstrate his brilliance behind the camera . xxmaj sadly it only xxunk in the final thirty minutes of the film , but if"


In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs = 16)

# Language Model Training, Testing and Interpretation

In [0]:
#Language Modeling is the task of Predicting the next word or character in a document.

In [12]:
data_lm.show_batch()

idx,text
0,"savage killer , melting every step of the way , he is xxunk by his friend , xxmaj dr. xxmaj xxunk xxmaj nelson ( xxmaj burr debenning ) . \n \n xxmaj this is often so xxunk funny - with enough absurd lines and situations to go around - that it 's hard for me to believe that the laughs are all xxunk . xxmaj it seems to me"
1,"xxmaj one becomes a character in a xxmaj king story they read , whereas xxmaj one is limited to petty xxunk of that same character when filmed . xxmaj for as long as xxmaj king writes , xxmaj hollywood will try shooting everything that comes out of his word xxunk , without any regard to whether or not they should . i do n't blame the filmmakers for trying ,"
2,"happen and i had to agree . xxmaj the xxmaj scene xxmaj that xxmaj explains xxmaj all was adequate and managed to explain all of the questions and mysterious dialogue bits throughout the movie but we were just checking them off a list . ( "" xxmaj oh , okay , that 's why xxmaj brad had that happen and xxmaj jonathan says this and ... "" ) \n \n"
3,"send this xxmaj uli xxmaj xxunk guy back to xxup movie xxup school . xxmaj who ever told him xxup he knew xxup how to make a movie ? xxmaj can just xxup anybody make movies these days ? xxmaj in the past , it always xxup required xxup talent before someone could make a movie . xxmaj after watching this lame xxup btk movie and the others he 's"
4,"training xxmaj day or xxmaj king xxmaj arthur , you will be impressed with the birth of this director in his second film xxmaj bait . xxmaj while xxmaj foxx gives a horrid , xxunk performance , there are certain scenes , which define xxmaj fuqua and demonstrate his brilliance behind the camera . xxmaj sadly it only xxunk in the final thirty minutes of the film , but if"


In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult = 0.5)
#AWD_LSTM stands for Averaged stocastic gradient descent with Weight Dropped LSTM.
  #Drop Connect
    #In Feed Forward and CNN's it is easier to use Dropout but in the case of RNN's we can't do that to hidden states becauses that hinders the ability of LSTM's long Term Dependencies.
    #So we use Dropout on the weights connecting the hidden states rather than on the hidden states and cell states.
  #ASGD
    #It has been found that for the particular task of language modeling, traditional SGD without momentum outperforms other algorithms such as momentum SGD, Adam, Adagrad, and RMSProp.
    #Therefore, the authors investigate a variant of the traditional SGD algorithm known as ASGD (Average SGD).
    #It takes gradient update step identical to the SGD algorithm but instead of returning the weight computed in the current iteration, it also takes into account the weights of the previous iterations and returns an average.

In [14]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.501260,3.961216,0.285699,11:38


In [0]:
learn.unfreeze()

In [16]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.159203,3.868247,0.293229,17:56


In [0]:
#That Accuracy is a good one as it is a language model.
#Our Pretrained Model is trained on WikiText-103 Dataset.
#On GPT-2 by using WikiText-103 the test perplexity = 17.48 
#On Transformer-XL Large by using WikiText-103 the test perplexity = 18.3

#Perplexity:
  #Perplexity is a measurement of how well a probability distribution or probability model predicts a sample.
  #It may be used to compare probability models.
  #A low perplexity indicates the probability distribution is good at predicting the sample.

# Prediction - Inference

In [60]:
learn.predict('The Movie', n_words = 10)

'The Movie Preview started running over box office at a church'

In [0]:
learn.save_encoder('ft_enc')

# Classifier Training, Testing and Interpretation.

In [0]:
#Sentiment Analysis is the task of Classifying the Polarity of a given Text.

In [63]:
data_clas.show_batch()

text,target
"xxbos xxmaj xxunk xxmaj victor xxmaj xxunk : a xxmaj review \n \n xxmaj you know , xxmaj xxunk xxmaj victor xxmaj xxunk is like sticking your hands into a big , xxunk xxunk of xxunk . xxmaj it 's warm and xxunk , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and xxunk xxmaj xxunk xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the format is the same as xxmaj xxunk xxmaj xxunk ' "" xxmaj la xxmaj xxunk",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive


In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult = 0.5)

In [0]:
learn.load_encoder('ft_enc')

In [66]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.573762,0.546005,0.766169,08:53


In [0]:
learn.freeze_to(-2)

In [68]:
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.537411,0.473649,0.781095,12:06


In [0]:
learn.unfreeze()

In [70]:
learn.fit_one_cycle(1, slice(2e-3/100., 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.466473,0.449686,0.855721,37:04


In [82]:
learn.predict("This is a good movie")

(Category positive, tensor(1), tensor([0.0043, 0.9957]))

In [0]:
learn.save('stage-1')

In [0]:
#We achieved an Accuracy of 85.5%.